In [189]:
# Lucas Xu, Highschool Intern @ UAlberta
#Python 3.11.9, WindowsOS 11 
#UI of a heatmap designed to visualize the amount of people per room
#main var/classes: class PannableCanvas, root, button_zoomin, button_zoomout, slider
import tkinter as tk
import tkinter.font as tkFont
from PIL import Image, ImageTk, ImageDraw
from tkinter import filedialog
from tkinter import simpledialog
from tkinter import messagebox
from tkinter import ttk
import math
import fitz
import csv
import json
import pandas as pd
import os
import tempfile
import shutil
import numpy as np
from enum import Enum
from shapely.geometry import Point, Polygon # for point polygon intersection


In [190]:
# generate sample data: a csv with hour 0-24 and number of people

# import random
# SAMPLE_SIZE=5
# pd.DataFrame({
#     'id': range(SAMPLE_SIZE),
#     'people': [[int((10 * random.random()) * np.sinc((i/12-1))) for i in range(24)] for _ in range(SAMPLE_SIZE)]
# }).to_csv('occupancy.csv', index=False)


In [191]:
# window init
root = tk.Tk()
root.title("HeatMap UI")
root.geometry("1600x1000")
root.config(bg='white', pady=100, padx=100)
# root.state('zoomed')
root.state('normal')

max_width = 900
max_height = 800

In [192]:
# file init stuff

#init filename database
filename_database = set()

#temp dir init
temp_dir = tempfile.mkdtemp()

#Have the path where the executable file will be when further implemented

image_path = "test.png"
data_path = "occupancy.csv"

# just the default that the file explorer will open to.
default_search_path = '/Users/stefan/Documents/life/work/network/localization/app/wifiscans/floormaps/Administration'


In [193]:
# make numpy print in json-friendly way
np.set_printoptions(formatter={'all': lambda x: f'{x:.2f}'})

In [194]:
# canvas params
snap_radius = 0.01 # in json coordinates 
# (so 0.01 is 1% of the width/height of the map. Ellipse shaped snap radius I guess)

min_zoom = 0.1
max_zoom = 10

In [195]:
# define Enum class
class DrawState(Enum):
    IDLE=0
    PAINTING=1
    ERASING=2

class ShiftState(Enum):
    NOT_SHIFTING=0
    SHIFTING=1

#define a canvas class
#Canvas init, canvas to hold a pannable image of the map
class PannableCanvas(tk.Frame):
    
    #define parameters and initialize
    def __init__(self, master, image_path, max_width, max_height, current_hour, data_path, *args, **kwargs):

        #init parent class holding canvas
        super().__init__(master,*args,**kwargs)

        #create class args to be made public
        self.image_path = image_path
        self.original_image = None # will be a PIL image
        self.image = None # will be a tk PhotoImage
        self.original_image_width = None
        self.original_image_height = None
        self.data_path = data_path
        self.canvas_width = max_width
        self.canvas_height = max_height
        self.current_scale = 1.0
        self.tk_image = None
        self.roomdata_path = None # set in display_selected()
        self.rooms = pd.DataFrame(columns = ['name', 'coords']) # index is id
        self.data = None
        self.memorystates = []
        self.current_hour = current_hour

        # shapes are the return values from ImageDraw.Draw().polygon(). stored so they can be erased later
        # lines are the return values from self.canvas.create_line. stored so they can be erased later
        # vertices are the raw coordinate data of the currently in-progress drawn verts
        
        self.shapes = [] # for erasing purposes
        self.lines = [] # for erasing purposes    
        self.mouseline = None # for erasing purposes
        self.previewline = None # for erasing purposes
        self.vertices = np.empty(shape=(0, 2)) # shape=(n_verts, 2) in json coordinate space
        
        self.draw_state = DrawState.IDLE
        self.shift_state = ShiftState.NOT_SHIFTING
        
        self.orientation = 1
        
        #canvas init
        self.canvas = tk.Canvas(self, width=max_width, height=max_height, bg='white')
        self.canvas.grid(row=0,column=0, sticky = "nsew")
        
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)
        
        #image init
        self.original_image, self.original_image_width, self.original_image_height = self.open_new_image(image_path)
        self.current_scale = self.calculate_ideal_scale(self.original_image, max_width, max_height)
        self.image = self.scale_image(self.original_image, self.current_scale)

        self.canvas_image = self.canvas.create_image(0,0, anchor=tk.NW, image=self.image)
        self.canvas.config(scrollregion=self.canvas.bbox(tk.ALL))
        
        #mouse init
        self.key_rebind()
        self.secondary_return_behaviour = False # if this is true, enter key does something different.
        self.right_clicked_shape_idx = None # if this is non-None, the user right clicked this shape and the next Return key should delete the image.
        self.hovered_room_idx = None # this is the idx into self.rooms (and self.data) of which room is hovered
        self.last_hover_event = None # this is the event from the most recent Motion B1 event
        
        #center canvas
        if self.image is not None:
            self.center_canvas()

    #a function wrapper to run 2 functions at once for event binding
    def on_mouse_motion(self, event):
        self.hover_over(event)
        self.draw_mouseline(event)

    #using the roomdata path, find current existing csv or create csv of rooms of that floor
    def extract_rooms(self):
        print(f"extracting rooms from {self.roomdata_path}")
        df = pd.DataFrame(columns = ['id', 'name', 'coords'])
        try:
            maybe_valid = pd.read_csv(self.roomdata_path)
            if list(df.columns) != ['id', 'name', 'coords']:
                print("Invalid format. Not reading the file (it may get wiped very soon).")
            else:
                # looks good
                maybe_valid['coords'] = maybe_valid['coords'].apply(lambda x: np.array(json.loads(x.replace(' ', ','))))
                df = maybe_valid
                

        except:
            print("File not found in roomdata_path.")

        df.set_index('id', inplace=True)
        self.rooms = df
        self.update_image()

    #extract the data of how many people are in a room
    def extract_data(self):
        try:
            df = pd.read_csv(self.data_path)
        except:
            tk.messagebox.showwarning(title="File not Found", message="Invalid path to file, file not found.")
            return

        #'people' column will be a list, with each index representing an hour 0 = 0:00,  23 = 23:00
        if list(df.columns) != ['id', 'people']:
            tk.messagebox.showwarning(title="Invalid Format", message="File format is invalid, file not used.")
            return

        df['people'] = df['people'].apply(json.loads)
        df.set_index('id', inplace=True)
        self.data = df
        self.update_image()

    #rebind keys when paint mode is on/off
    def key_rebind(self):
        #include shift for later when grid snapping is worked on
        print(f"rebinding keys to {self.draw_state}")
        
        # common
        self.canvas.unbind("<ButtonPress-1>")
        self.canvas.unbind("<B1-Motion>")
        self.canvas.bind("<ButtonPress-1>", self.start_pan)
        self.canvas.bind("<B1-Motion>", self.pan)

        self.canvas.unbind('<ButtonPress-3>')
        self.unbind('<KeyRelease>')
        self.canvas.unbind("<Motion>")

        
        if self.draw_state == DrawState.PAINTING:
            #code for paint mode
            
            self.canvas.bind("<ButtonPress-3>", self.create_vertex)
            self.canvas.bind("<Motion>", self.on_mouse_motion)
            self.bind("<KeyRelease>", self.can_create)
            self.canvas.bind("<KeyPress>", self.shifting)
        elif self.draw_state == DrawState.ERASING:
            self.canvas.bind("<Motion>", self.hover_over)
            self.canvas.bind("<ButtonPress-3>", self.on_erase_mode_right_click)
            self.bind("<KeyRelease>", self.can_delete)
        elif self.draw_state == DrawState.IDLE:
            self.canvas.bind("<Motion>", self.hover_over)
        else:
            # this will be reached only if there's a bug
            raise Exception(f"Unhandled self.draw_state {self.draw_state}")

    # returns a new image from path image_path
    def open_new_image(self, image_path):

        image_path = image_path.strip().strip("'").strip('"')
        try:
            image = Image.open(image_path)
        except Exception as e:
            print(f"Error opening image: {e}")
            return None
            
        image_w, image_h = image.size
        
        return image, image_w, image_h

    # return best scale so that the image will perfectly fit onto the canvas.
    def calculate_ideal_scale(self, image, canvas_w, canvas_h):
        image_w, image_h = image.size
        
        scale_x = canvas_w / image_w
        scale_y = canvas_h / image_h

        # if either of scale_x or scale_y are < 1, then that means the image will not fit on the canvas.
        # need to find which is lesser and set the current_scale to that.
        lesser_of_the_two = min(scale_x, scale_y)

        # clamp it inbewteen min and max zoom.
        return min(max(lesser_of_the_two, min_zoom), max_zoom)
        
    # scales an image by a scalar scale
    def scale_image(self, image, scale):
        image_w, image_h = image.size
        scaled_size = (int(image_w * scale), int(image_h * scale))
        return ImageTk.PhotoImage(image.resize(scaled_size))

    # redraw image and all shapes for when zoomin/zoomout is called or hour change
    def update_image(self):
        self.image = self.scale_image(self.original_image, self.current_scale)
        if self.image:
            self.canvas.itemconfig(self.canvas_image, image=self.image)
            self.canvas.config(scrollregion=self.canvas.bbox(tk.ALL))
        else:
            print("No image loaded")
        
        self.draw_shapes()
        
        self.draw_vertices()

    #zoom in
    def zoom_in(self):
        if self.current_scale * 1.2 <= max_zoom:
            self.current_scale *= 1.2 
            self.update_image()
        print(f'current scale: {self.current_scale}')
    #zoom out
    def zoom_out(self):
        if self.current_scale / 1.2 >= min_zoom:
            self.current_scale /= 1.2
            self.update_image()
        print(f'current zoom: {self.current_scale}')
            
    # called when mousebutton1 is pressed down initially
    def start_pan(self, event):
        #track starting coords
        hide_tooltip()
        self.canvas.scan_mark(event.x, event.y)
        print(f'canvas-space coords: {self.convert_coords_window_to_canvas((event.x, event.y))}')

    # called when mousebutton 1 is dragged 
    def pan(self, event):
        # drag with scan_dragto
        self.canvas.scan_dragto(event.x, event.y, gain=1)


    # COORDINATE SPACES
    # window space: (0, 0) is top left of the screen, (screen_w,screen_h) is bottom right
    # canvas: (0, 0) is the top left of the canvas, (image_w,image_h) is bottom right of canvas
    # json: (0, 0) is the top left of the canvas, (1,1) is the bottom right of canvas
    
    def convert_coords_window_to_canvas(self, coords):
        x, y = coords
        return self.canvas.canvasx(x), self.canvas.canvasy(y)

    def convert_coords_canvas_to_json(self, coords):
        x, y = coords
        return x/self.original_image_width/self.current_scale, y/self.original_image_height/self.current_scale

    def convert_coords_json_to_canvas(self, coords):
        x, y = coords
        return x*self.original_image_width*self.current_scale, y*self.original_image_height*self.current_scale

    # shortcuts from window straight to json and back
    def convert_coords_window_to_json(self, coords):
        return self.convert_coords_canvas_to_json(self.convert_coords_window_to_canvas(coords))

    def snap_to_nearest_vertex(self, coords):
        x, y = coords
        np_vertices = np.array(self.vertices)

        if len(self.rooms['coords']) > 0:
            np_savedvertices = np.concatenate(self.rooms['coords'].values)
        else:
            np_savedvertices = np.array([])
        
        # acutally distance squared for performance but whateva
        distance_to_point = {}
        
        for vx, vy in np.append(np_vertices, np_savedvertices).reshape(-1,2):
            dx = vx - x 
            dy = vy - y
            dist_squared = dx**2 + dy**2
            if dist_squared < (snap_radius)**2:
                distance_to_point[vx, vy] = dist_squared
        
        return min(distance_to_point, key=distance_to_point.get, default=coords)
    
    
    def create_vertex(self, event):

        json_xy = self.convert_coords_window_to_json((event.x, event.y))

        #auto snap, check if current x,y is close to any x,y in the list
        if self.shift_state == ShiftState.NOT_SHIFTING:
             snapped_xy = self.snap_to_nearest_vertex(json_xy)
        
        # vstack will just append the snapped vertices to the vertices np.array by creating a new row.
        self.vertices = np.vstack((self.vertices, snapped_xy))

        print(f"creating vertex at {snapped_xy} new num vertices {len(self.vertices)}")
        
        self.update_image()

    def draw_vertices(self):

        # erase all the existing lines
        # TODO maybe instead draw once and just update the one which moves with the cursor.
        for line in self.lines:
            self.canvas.delete(line)
        self.lines.clear()
            
        # scale the new coordiantes based off the scale
        canvas_vertex_coords = [self.convert_coords_json_to_canvas(coords) for coords in self.vertices]

        # define this lambda since its only used in this one function.
        # returns [] if len(
        sliding_window = lambda data, window_size, step_size: [data[i:i + window_size] for i in range(0, len(data) - window_size + 1, step_size)]

        #connect vertices from one to another in the list, from the bottom to top
        for start_vertex, end_vertex in sliding_window(canvas_vertex_coords, 2, 1):
            start_x, start_y = start_vertex
            end_x, end_y = end_vertex

            # draw the line
            # save it to a list so it can be erased later
            line = self.canvas.create_line(start_x, start_y, end_x, end_y, fill='black', width=4)
            self.lines.append(line)

        # check if there are 3 vertices and if so draw the previewline.
        if len(self.vertices) >= 3:
            start_xy = self.convert_coords_json_to_canvas(self.vertices[-1])
            end_xy = self.convert_coords_json_to_canvas(self.vertices[0])        
            
            #clear previous line
            self.canvas.delete(self.previewline)
    
            # draw new line
            start_x, start_y = start_xy
            end_x, end_y = end_xy
            self.previewline = self.canvas.create_line(start_x, start_y, end_x, end_y, fill="pink", dash=(10, 4), width=4)

            show_tooltip(self.last_hover_event, "Close shape [Return]")

    #while actively drawing a box, connect the last drawn vertex to mouse
    def draw_mouseline(self, event):
        if self.vertices.size == 0:
            return
            
        start_xy = self.convert_coords_json_to_canvas(self.vertices[-1])
        end_xy = self.convert_coords_window_to_json((event.x, event.y))

        if self.shift_state == ShiftState.NOT_SHIFTING:
            end_xy = self.snap_to_nearest_vertex(end_xy)

        end_xy = self.convert_coords_json_to_canvas(end_xy)
        
        #clear previous line
        self.canvas.delete(self.mouseline)

        #draw new line
        start_x, start_y = start_xy
        end_x, end_y = end_xy
        self.mouseline = self.canvas.create_line(start_x, start_y, end_x, end_y, fill="black", width=2)

    # raycast algorithm to detect whether mouse is in a shape
    # param xy_json is in json coordinate space.
    # returns None if no intersections or the INDEX in self.rooms of the clicked-on room 
    def raycast_detect(self, xy_json):
        
        point = Point(xy_json)
        
        for index, coords in self.rooms['coords'].items():

            polygon = Polygon(coords.reshape(-1, 2))

            if polygon.contains(point):
                return index

        return None


    def set_right_clicked_shape_idx(self, idx):
        self.right_clicked_shape_idx = idx

    def on_erase_mode_right_click(self, event):
        
        xy_json = self.convert_coords_window_to_json((event.x, event.y))
        self.set_right_clicked_shape_idx(self.raycast_detect(xy_json))

        if self.right_clicked_shape_idx is not None:
            
            # if a tooltip already exists, just hide it. otherwise open a new tooltip.
            show_tooltip(event, "Delete shape? [Return]", lambda: self.set_right_clicked_shape_idx(None))
        
    def can_delete(self, event):
        if event.keysym == 'Return':
            if(self.set_right_clicked_shape_idx is not None):
                self.rooms.drop(self.right_clicked_shape_idx, inplace=True)
                self.update_image()  
                hide_tooltip()

        if event.keysym == 'Escape':
            hide_tooltip()

    def update_label(self, event):
        global label

        if self.hovered_room_idx is not None:
            txt_roomname = self.rooms.loc[self.hovered_room_idx, 'name']
            txt_roomid = self.hovered_room_idx

            if self.data is not None and self.hovered_room_idx in self.data.index:
                txt_roomppl = self.data.loc[self.hovered_room_idx, 'people'][self.current_hour]
            else:
                txt_roomppl = "missing data file."

            show_tooltip(event, f"name:{txt_roomname}\nid:{txt_roomid}\noccupancy:{txt_roomppl}")
        else:
            hide_tooltip()
            
    
    #detect whether the mouse is hovered over a shape using ray casting, with hover, display room id and # of ppl, in delete mode, right clicking will delete
    def hover_over(self, event):
        self.last_hover_event = event

        if self.image:    
            xy_json = self.convert_coords_window_to_json((event.x, event.y))
            self.hovered_room_idx = self.raycast_detect(xy_json)

            # 1 second after finishing hovering, show the info
            global after_id_2

            if after_id_2 is not None:
                root.after_cancel(after_id_2)
                after_id_2 = None

            # only show this tooltip in drawing state.
            if self.draw_state == DrawState.IDLE:
                after_id_2 = root.after(1000, lambda: self.update_label(self.last_hover_event))
            
            

    def set_secondary_return_behaviour(self, state):
        self.secondary_return_behaviour = state
    
    #check if shape can be created based off key-release
    def can_create(self, event):
        if event.keysym == 'Return':
            if self.secondary_return_behaviour:
                self.delete_in_progress_shape()
                hide_tooltip()
            elif len(self.vertices) >= 3: # need at least a triangle (3 verts) to make a shape
                self.create_shape()
        #also detect if the keyrelease is left shift to make is_shifting false, as this is a keyrelease detect funtion
        elif event.keysym == 'Shift_L':
            self.shift_state = ShiftState.NOT_SHIFTING

        elif event.keysym == 'Escape':

            # if there wasn't a tooltip already, make a tooltip. otherwise just close it and do nothing.
            if not hide_tooltip():
                show_tooltip(event, "Delete in progress shape [Return]", lambda: self.set_secondary_return_behaviour(False))
                self.set_secondary_return_behaviour(True) # will turn off on tooltip hide
        
    def shifting(self, event):
        if event.keysym == 'Shift_L':
            self.shift_state = ShiftState.SHIFTING

    # prompt is the text to display to the user when asking for user input
    # condition is a lambda String: Boolean. input is the user provided string and 
    # output is true if the input is good and false if the function should continue
    # asking for user input.
    # message - the message to show if the output of parameter condition is false.
    # returns the user's input or None if user cancels.
    def ask_for_user_input_until_valid(self, prompt, condition=lambda x: True, on_fail_message="Debug: Custom condition failed."):
        user_input = None
        while user_input is None:
            user_input = simpledialog.askstring("Input", prompt)
            if user_input is None:  # The user canceled the input
                messagebox.showwarning("Warning", "Operation cancelled")
                return None
            elif not user_input.strip():  # The user entered an empty string
                messagebox.showwarning("Warning", "Input cannot be blank!")
                user_input = None
            elif not condition(user_input):
                messagebox.showwarning("Warning", on_fail_message)
                user_input = None

        return user_input

    # deletes and erases the currently drawn shape
    def delete_in_progress_shape(self):
        # delete vertices memory
        self.vertices = np.empty(shape=(0, 2))
            
        # erase all lines from UI
        for line in self.lines:
            self.canvas.delete(line)
                
        # erase lines memory
        self.lines.clear()

        # erase mouse line
        self.canvas.delete(self.mouseline)
        self.canvas.delete(self.previewline)
    
    #take current placed vertices and create a shape that can be pushed into csv and pandas df for other usages
    def create_shape(self):

        # ask for room id. Additionally, if it already exists in the rooms, ask again.
        room_id = self.ask_for_user_input_until_valid(
            "Enter room id:",
            lambda x: string_is_int(x) and int(x) not in self.rooms.index.values, # must convert the rooms[id] to string in case they are all ints.
            "Already existing input or input is not an int. Please enter a different id"
        )

        # user cancelled the thing.
        if room_id == None:
            self.delete_in_progress_shape()
            return

        # ask for room name. Duplicates allowed so no need to check the dataframe.
        room_name = self.ask_for_user_input_until_valid("Enter room name:")

        # user cancelled the 
        if room_name == None:
            return
 
        self.rooms = pd.concat([self.rooms, pd.DataFrame({
                                    'name': [room_name], 
                                    'coords': [self.vertices.flatten()]},
                                    index=[room_id])])

        self.rooms.reset_index(drop=True, inplace=True)
    
        self.delete_in_progress_shape()
        
        self.update_image()

        # autosave!!!
        self.save_file()

        #calculate colour of shape from green to red based off of how many people
    def calc_color(self, idx):
        
        default_red = 255, 0, 0
        
        #scale of 1-10, 10 being max power
        if self.data is None or self.data.empty:
            return default_red

        # get the row with 
        try:
            row = self.data.loc[idx]

            if row.empty:
                return default_red

        except KeyError as e:
            return default_red

        occupancy = row['people'][self.current_hour]
            
        val = min(occupancy/10, 1)
        
        red = int((1-val) * 255)
        green = int(val * 255)
        return red, green, 0

    
    #draw all rooms of the current hour to be overlayed on the image using coordinates and draw.polygon()
    #going to need another parameter, "people" to take in a pandas object to associate # of ppl per room
    def draw_shapes(self):

        # erase old shapes
        for shape in self.shapes:
            self.canvas.delete(shape['id'])

        self.shapes.clear()

        if self.rooms is None or self.rooms.empty:
            return

        # draw new shapes
        for index, room in self.rooms.iterrows():
            # #init values for polygon
            r, g, b = self.calc_color(index)
            alpha = 128
            coords = [self.convert_coords_json_to_canvas(coords) for coords in room['coords'].reshape(-1, 2)]
            
            #create image with RGBA values for translucent effect
            img = Image.new('RGBA', (self.original_image_width, self.original_image_height), (0, 0, 0, 0))
            draw = ImageDraw.Draw(img)
    
            #fill the image with a polygon of calculated colour
            #polygons by default do not take alpha values, hence the image was created
            
            draw.polygon(coords, fill=(r, g, b, alpha), outline='black')
                
            tk_img = ImageTk.PhotoImage(img)
    
            image_item = self.canvas.create_image(0, 0, anchor=tk.NW, image=tk_img)
    
            #add shapes into list for easy access when updating
            self.shapes.append({'id': image_item, 'original_coords': coords, 'image': tk_img})

    #when finished with paint mode, save the pandas dataframe into a csv file in roomdata_path
    def save_file(self):
        self.rooms.to_csv(self.roomdata_path, index=True, index_label='id')
    
    #center the canvas
    def center_canvas(self):
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        image_width = self.image.width()
        image_height = self.image.height()

        if image_width > canvas_width:
            self.canvas.xview_moveto((image_width - canvas_width) / 2 / image_width)
        else:
            self.canvas.xview_moveto(0.5 - canvas_width / image_width / 2)

        if image_height > canvas_height:
            self.canvas.yview_moveto((image_height - canvas_height) / 2 / image_height)
        else:
            self.canvas.yview_moveto(0.5 - canvas_height / image_height / 2)

        # Adjust the scroll region to accommodate zoom
        self.canvas.config(scrollregion=self.canvas.bbox(tk.ALL))


In [196]:
#helper functions (they operate on the global objects)

def string_is_int(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

def zoom_out():
    canvas.zoom_out()

def zoom_in():
    canvas.zoom_in()

#update value of the slider to show the data of specified hour
def slider_change(value):
    canvas.current_hour = int(value)
    
    if canvas.last_hover_event is not None and canvas.draw_state == DrawState.IDLE:
        canvas.update_label(canvas.last_hover_event)
    
    canvas.update_image()

#function for changing the hour for button_change()
def change_hour():
    global after_id
    current_val = slider.get()
    if current_val == 23:
        current_val = 0
    if button_video.image == pause_img:
        if current_val + 1 < 24:
            current_val += 1
            slider.set(current_val)
            canvas.update_image()
            after_id = root.after(1000, change_hour)
        else:
            return
  
#play pause function for play pause button
def button_change():
    global after_id
    #if the images are currently paused
    if button_video.image == play_img:
        button_video.config(image = pause_img)
        button_video.image = pause_img
        change_hour()
    else:
        #if the iamges are currently playing
        button_video.config(image = play_img)
        button_video.image = play_img
        if after_id is not None:
            root.after_cancel(after_id)  # Cancel the scheduled increment
            after_id = None

#rotating the image for the rotate button
def button_rotate():
    #delete later, currently cannot solve rotation for the life of me
    if canvas.image:
        canvas.orientation += 1
        if canvas.orientation > 4:
            canvas.orientation = 1
        canvas.update_image()

#update the listbox
def update_listbox(data):
    #first clear the listbox
    listbox.delete(0, tk.END)

    #upload new data
    for item in data:
        listbox.insert(tk.END, item)


#button function to upload directoy
def upload_dir():
    dirname_data = filedialog.askdirectory(title="Select a directory", initialdir=default_search_path)
    if not dirname_data:
        return

    #walk through all sub directories and get to files
    for root, dirs, files in os.walk(dirname_data):
        for file in files:
            file_path = os.path.join(root, file)
            if file.lower().endswith('pdf'):
                #Potential issue here with fitz.open not using the full absolute path, instead it is only using the name
                pdf = fitz.open(file)
                for i in range(len(pdf)): 
                #save output of jpg
                    page = pdf.load_page(i)
                    img = page.get_pixmap()
                    newpdf_name = f'page{i+1}_{name}.png'
                    newpdf_path = os.path.join(temp_dir, newpdf_name)
                    img.save(newpdf_path)
                    filename_database.add(newpdf_path)
            elif file.lower().endswith('png'):
                 filename_database.add(file_path)

    #update listbox
    update_listbox(filename_database)

#requests a file destination input from user, stores path into database
def upload_image():
    filename_data = filedialog.askopenfilename(title="Select a file", filetypes=(("PNG", "*.png"), ("PDF", "*.pdf")))
    if not filename_data:
        return
    
    #check if pdf
    _, extension = os.path.splitext(filename_data)
    
    if extension == ".pdf":
        #convert from pdf into jpg
        pdf = fitz.open(filename_data)
        for i in range(len(pdf)): 
            #save output of jpg
            page = pdf.load_page(i)
            img = page.get_pixmap()
            newpdf_name = f'page{i+1}_{name}.png'
            newpdf_path = os.path.join(temp_dir, newpdf_name)
            img.save(newpdf_path)
            filename_database.add(newpdf_path)

    else:
        filename_database.add(filename_data)

    update_listbox(filename_database)

#update the listbox based on what's currently inside the search box
def search_filter(event):
    value = searchbox.get()

    if value == '':
        data = filename_database
    else:
        data = []

        #loop through all items of the database and add what has value in it
        for item in filename_database:
            if value.lower() in item.lower():
                data.append(item)

    #send the new list to listbox
    update_listbox(data)

#uplaod the csv data to assign id numbers to how many people
def upload_data():
    filename_data = filedialog.askopenfilename(title="Select a file", filetypes=(("CSV", "*.csv"),))
    canvas.data_path = filename_data
    canvas.extract_data()

#grab selection in combobox, and then display on canvas
def display_selected(event):
    global listbox
    global canvas
    #grab the filename from combobox
    option_name = listbox.get(listbox.curselection())
    #using filename as a key, get filename's path
    #set canvas roomdata path to represent the combobox selection
    canvas.roomdata_path = os.path.splitext(option_name)[0]+'.csv'
    canvas.extract_rooms()
    canvas.image_path = option_name
    
    canvas.original_image, canvas.original_image_width, canvas.original_image_height = canvas.open_new_image(option_name)
    canvas.current_scale = canvas.calculate_ideal_scale(canvas.original_image, canvas.canvas_width, canvas.canvas_height)
    canvas.update_image() # sets canvas.image <- scale_image(original_image, current_scale)
    
    canvas.center_canvas()
    canvas.pack(fill=tk.BOTH, expand=True)

def defocus(event):
    event.widget.master.focus_set()

#window close protocols
def on_close():

    global after_id
    if after_id is not None:
        root.after_cancel(after_id)
    global after_id_2
    if after_id_2 is not None:
        root.after_cancel(after_id_2)
    
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        root.destroy()


In [197]:
#define a bunch of global ui elements

#frame and button init
top_frame = tk.Frame(root, bg = 'light gray')
top_frame.pack(side = tk.TOP)
top_left = tk.Frame(top_frame, bg='light blue')
top_left.pack(side = "left")

# rotate_path = Image.open(r'rotate-left.png')
# rotate_path = rotate_path.resize((30,30), Image.LANCZOS)
# rotate_img = ImageTk.PhotoImage(rotate_path)
# button_rotate = tk.Button(top_frame, image=rotate_img,  width=35, height=35, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=button_rotate)
# button_rotate = tk.Button(top_frame,  text="rotate", width=35, height=35, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=button_rotate)
# button_rotate.pack(side = 'right', padx=10, pady=10)

button_zoomin = tk.Button(top_frame, text = "+", width=4, height=2, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=zoom_in)
button_zoomin.pack(side = 'right', expand = False, pady=10)
button_zoomout = tk.Button(top_frame, text = "-", width=4, height=2, relief='solid',bg='white', activebackground='white', activeforeground='black',  borderwidth=1, highlightbackground = 'grey', command=zoom_out)
button_zoomout.pack(side = 'right', expand = False, pady=10)

right_frame = tk.Frame(root, bg = 'light gray')
right_frame.pack(side = tk.RIGHT, padx = 10)
right_bottom = tk.Frame(right_frame, bg = 'light blue')
right_bottom.pack(side="bottom")

#Searchbox
searchbox = tk.Entry(right_frame, font=("Arial", 12), background = '#DDDDDD')
searchbox.pack(side= 'top', padx=10)
searchbox.bind('<KeyRelease>', search_filter)

#Listbox for searched values
listbox = tk.Listbox(right_frame, width = 30)
listbox.pack(side = 'top', padx=10, pady=10, expand= True)
listbox.bind('<<ListboxSelect>>', display_selected)
#combobox for map selection
# combobox = ttk.Combobox(right_frame)
# combobox.bind("<<ComboboxSelected>>", display_selected)
# combobox.pack(side = 'top', padx=10)

#scrollable listbox 
# scrollbox = create_scrollbox(right_frame)


# draw state radio selector

def radio_button_callback(draw_state):
    if draw_state == DrawState.PAINTING:
        # not entirely sure why but only key releases need focus every thing else seems to work ok.
        canvas.focus_set()
        
    canvas.draw_state = draw_state
    canvas.key_rebind()    

draw_state_int = tk.IntVar(value=DrawState.IDLE.value) # variable to hold state of radio. default it to IDLE

radio1 = tk.Radiobutton(right_bottom, text="Idle", variable=draw_state_int, value=DrawState.IDLE.value, command=lambda: radio_button_callback(DrawState.IDLE))
radio1.pack()
radio2 = tk.Radiobutton(right_bottom, text="Paint", variable=draw_state_int, value=DrawState.PAINTING.value, command=lambda: radio_button_callback(DrawState.PAINTING))
radio2.pack()
radio2 = tk.Radiobutton(right_bottom, text="Erase", variable=draw_state_int, value=DrawState.ERASING.value, command=lambda: radio_button_callback(DrawState.ERASING))
radio2.pack()

#load images for play/pause button
play_path = Image.open(r'play.png')
play_path = play_path.resize((35,35), Image.LANCZOS)
pause_path = Image.open(r'pause.png')
pause_path = pause_path.resize((35,35), Image.LANCZOS)
play_img = ImageTk.PhotoImage(play_path)
pause_img = ImageTk.PhotoImage(pause_path)
# button_video = tk.Button(top_frame, image=play_img, width=35, height=35, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=button_change)
button_video = tk.Button(top_frame, text="video", width=35, height=35, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=button_change)
button_video.image = play_img
button_video.pack(side = 'left', pady=10)

#upload dir with images
uploaddir_button = tk.Button(top_frame, text="Upload Directory", width=13, height=2, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=upload_dir)
uploaddir_button.pack(side = "left", pady=10, padx =10)

#Upload image button
uploadimg_button = tk.Button(top_frame, text="Upload Image", width=13, height=2, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=upload_image)
uploadimg_button.pack(side = "left", pady=10, padx =10)

#upload room ppl data
uploaddata_button = tk.Button(top_frame, text="Upload Data", width=13, height=2, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=upload_data)
uploaddata_button.pack(side= 'left', pady=10)

slider = tk.Scale(top_frame, from_ = 0, to=23, orient='horizontal', command=slider_change, length = 300)
slider.pack(pady=20, padx = 20)

hour = 0

tooltip = None  # Initialize tooltip variable

def _do_nothing(): pass
    
_on_tooltip_hide = _do_nothing
def show_tooltip(event, text, on_tooltip_hide = _do_nothing):
    global tooltip
    global _on_tooltip_hide

    # hide old tooltip.
    hide_tooltip()
    _on_tooltip_hide = on_tooltip_hide
    
    tooltip = tk.Toplevel(root)
    tooltip.wm_overrideredirect(True)
    tooltip.wm_geometry(f"+{event.x_root}+{event.y_root}")

    label = tk.Label(tooltip, text=text, bg="light gray", relief="solid", padx=5, pady=5)
    label.pack()

# returns true if the tooltip was successfully hidden and false if there was no tooltip to begin with.
def hide_tooltip():
    global tooltip
    global _on_tooltip_hide
    if tooltip is not None:
        tooltip.destroy()
        _on_tooltip_hide()
        _on_tooltip_hide = _do_nothing
        tooltip = None
        return True # it did exists and we deleted it
    else:
        return False # it doesn't exist

#canvas init
canvas = PannableCanvas(root, image_path, max_width, max_height, hour, data_path)
canvas.pack(fill=tk.BOTH, expand=True)

after_id_2 = None # jsut for default. this is like a timer id.

slider.config(resolution= 1)

rebinding keys to DrawState.IDLE


In [198]:

#window close protocols
root.protocol("WM_DELETE_WINDOW", on_close)

# Run the main event loop
root.mainloop()

rebinding keys to DrawState.PAINTING
creating vertex at (0.6725, 0.8375) new num vertices 1
creating vertex at (0.44875000000000004, 0.56125) new num vertices 2
creating vertex at (0.23875000000000002, 0.7750000000000001) new num vertices 3
creating vertex at (0.4625, 0.89125) new num vertices 4
creating vertex at (0.8612500000000001, 0.5637500000000001) new num vertices 5
creating vertex at (0.8750000000000001, 0.9175000000000001) new num vertices 6
creating vertex at (0.61125, 0.97125) new num vertices 7
creating vertex at (0.12000000000000001, 0.9525) new num vertices 8
creating vertex at (0.00625, 0.97) new num vertices 9
creating vertex at (0.007500000000000001, 0.41) new num vertices 10
canvas-space coords: (189.0, 358.0)
canvas-space coords: (329.0, 175.0)
creating vertex at (-0.0012500000000000002, -0.0012500000000000002) new num vertices 11
creating vertex at (0.66625, 0.4975) new num vertices 12
current zoom: 1.1111111111111112
current zoom: 0.925925925925926
current zoom: 0.

In [199]:
arr = np.empty((0,2))
arr = np.vstack((arr, (0.0, 0.0)))
arr = np.vstack((arr, (2.0, 0.0)))
arr = np.vstack((arr, (2.0, 2.0)))
arr = np.vstack((arr, (0.0, 2.0)))
x = [canvas.convert_coords_json_to_canvas(coords) for coords in arr]

sliding_window = lambda data, window_size, step_size: [data[i:i + window_size] for i in range(0, len(data) - window_size + 1, step_size)]

sliding_window(x, 2, 1)

[[(0.0, 0.0), (535.8367626886147, 0.0)],
 [(535.8367626886147, 0.0), (535.8367626886147, 535.8367626886147)],
 [(535.8367626886147, 535.8367626886147), (0.0, 535.8367626886147)]]